In [225]:
%load_ext autoreload
%autoreload 2

import cvxpy as cp
import numpy as np

import torch
from torch.nn.parameter import Parameter
import torch.optim as optim

import matplotlib.pyplot as plt
from tqdm import tqdm
from numpy import genfromtxt

%matplotlib inline

from controllers import DeePC, DDeePC
plt.rcParams["figure.figsize"] = (15,5)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [226]:
ud = genfromtxt('recht_ud.csv', delimiter=',')
yd = genfromtxt('recht_yd.csv', delimiter=',')

In [227]:
A = np.array([[1.01, 0.01, 0.00], # A - State-space matrix
              [0.01, 1.01, 0.01], 
              [0.00, 0.01, 1.01]]) 
B = np.eye(3) # B - State-space matrix
C = np.eye(3) # C - State-space matrix
D = np.zeros([3,3]) # D - State-space matrix
n = A.shape[0] # n = number of states
m = B.shape[1] # m = number of inputs
p = C.shape[0] # p = number of output
q = m+p # q = number of i/o variables

Tini = 4                                   # Past time horizon                                           
Tf = 10                                     # Future time horizon         
T = (m+1)*(Tini + Tf + n) - 1    
noise_std = 0.1                       

In [228]:
# yd += np.random.rand(T,p)*noise_std
u_ini = ud[:Tini,:].reshape((Tini*m,))
y_ini = yd[:Tini,:].reshape((Tini*p,))
y_constraints = np.kron(np.ones(Tf), np.array([100,100,100]))
u_constraonts = np.kron(np.ones(Tf), np.array([20,20,20]))

In [229]:
q = torch.ones(3)*1.8
r = torch.ones(3)*0.018
lam_y = torch.Tensor([0.125])

# expert = DDeePC(
#     ud=ud, yd=yd, N=Tf, Tini=Tini, T=T, p=3, m=3,
#     y_constraints=y_constraints, u_constraints=u_constraonts,
#     stochastic=True, linear=True, q=q, r=r, lam_y=lam_y
# )

learner = DDeePC(
    ud=ud, yd=yd, N=Tf, Tini=Tini, T=T, p=3, m=3,
    y_constraints=y_constraints, u_constraints=u_constraonts,
    stochastic=True, linear=True, q=q, r=r
)

In [230]:
for param in learner.parameters():
    print(param)

Parameter containing:
tensor([1.0068], requires_grad=True)


In [234]:
yd.shape

(67, 3)

In [231]:
optim = torch.optim.Adam(learner.parameters(), lr=1e-2)
pbar = tqdm(range(2000), ncols=120)
criterion = torch.nn.MSELoss()

for i in pbar:
    
    ref = torch.Tensor(np.random.uniform(size=(3,), low=-2.0, high=2.0))
    ref = torch.kron(torch.ones(Tf), ref)
    
    index = int(np.random.uniform(low=0, high=50))
    u_pred, y_pred , cost = learner(ref=ref, u_ini=torch.Tensor(u_ini), y_ini=torch.Tensor(y_ini + np.random.randn(Tini*p,)*noise_std))

    loss = criterion(input=y_pred, target=ref)
    optim.zero_grad()
    loss.backward()
    optim.step()
    
    pbar.set_description(f'Loss = {loss.item():.7f}, lam_y = {learner.lam_y.item():.4f}')

Loss = 0.0000482, lam_y = 0.2171: 100%|█████████████████████████████████████████████| 2000/2000 [01:31<00:00, 21.91it/s]


In [232]:
for param in learner.parameters():
    print(param)

Parameter containing:
tensor([0.2171], requires_grad=True)
